# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.clustering.kcenters.scala.KMeans
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.indices.ExternalIndicesLocal
import org.clustering4ever.enums.NmiNormalizationNature._
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.vectors.ScalarVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.clustering.kcenters.scala.KMeans
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.indices.ExternalIndicesLocal
import org.clustering4ever.enums.NmiNormalizationNature._
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.vectors.ScalarVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-01-13 11:02:30--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.4’

     0K .......                                               100% 21.6M=0s

2019-01-13 11:02:30 (21.6 MB/s) - ‘/tmp/aggregation.csv.4’ saved [8063/8063]

:sh: Scheme missing.
--2019-01-13 11:02:30--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-01-13 11:02:30--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.4’

     0K .                                                     100

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 100000
val dim = 10
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => EasyClusterizable(id.toLong, new ScalarVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 100000
dim: Int = 10
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(15.55, 28.65), ArrayBuffer(14.9, 27.55), ArrayBuffer(14.45, 28.35), ArrayBuffer(14.15, 28.8), ArrayBuffer(13.75, 28.05), ArrayBuffer(13.35, 28.45), ArrayBuffer(13.0, 29.15), ArrayBuffer(13.45, 27.5), ArrayBuffer(13.6, 26.5), ArrayBuffer(12.8, 27.35), ArrayBuffer(12.4, 27.85), ArrayBuffer(12.3, 28.4), ArrayBuffer(12.2, 28.65), ArrayBuffer(13.4, 25.1), ArrayBuffer(12.95, 25.95), ArrayBuffer(12.9, 26.5), ArrayBuffer(11.85, 27.0), ArrayBuffer(11.35, 28.0), ArrayBuffer(11.15, 28.7), ArrayBuffer(11.25, 27.4), ArrayBuffer(10.75, 27.7), ArrayBuffer(10.5, 28.35), ArrayBuffer(9....

## Parameters 

In [ ]:
val k = 7
val iterMax = 50
val epsilon = 0.001
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](false)
// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](4)

k: Int = 7
iterMax: Int = 50
epsilon: Double = 0.001
metric1: org.clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without squared root applied
metric2: org.clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = Minkowski(4,3)


## Run and measure the algorithm time

In [ ]:
// val data = parData.seq

val t1 = System.nanoTime

val model = KMeans.run(parData, k, metric1, iterMax, epsilon)

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 30813978293569
model: org.clustering4ever.clustering.kcenters.scala.KMeansModel[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.clusterizables.EasyClusterizable,org.clustering4ever.math.distances.scalar.Euclidean,scala.collection.parallel.mutable.ParSeq] = KMeansModel(Map(0 -> ScalarVector(ArrayBuffer(19.040686274509802, 7.05906862745098)), 5 -> ScalarVector(ArrayBuffer(9.41235294117647, 19.98411764705882)), 1 -> ScalarVector(ArrayBuffer(9.785401459854015, 7.606569343065693)), 6 -> ScalarVector(ArrayBuffer(21.543333333333333, 23.003333333333334)), 2 -> ScalarVector(ArrayBuffer(33.142788461538466, 8.793750000000001)), 3 -> ScalarVector(ArrayBuffer(9.323529411764707, 25....

0.18623518

In [ ]:
val rawModel = KMeans.run(rawData, k, metric1, iterMax, epsilon)

rawModel.centerPredict(rawData)

rawModel: org.clustering4ever.clustering.kcenters.scala.KMeansModel[Int,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.clusterizables.EasyClusterizable,org.clustering4ever.math.distances.scalar.Euclidean,scala.collection.mutable.Buffer] = KMeansModel(Map(0 -> ScalarVector(ArrayBuffer(33.142788461538466, 8.793750000000001)), 5 -> ScalarVector(ArrayBuffer(9.069343065693426, 24.139051094890508)), 1 -> ScalarVector(ArrayBuffer(9.370394736842101, 14.450000000000003)), 6 -> ScalarVector(ArrayBuffer(21.543333333333326, 23.003333333333334)), 2 -> ScalarVector(ArrayBuffer(19.255927835051548, 7.2564432989690735)), 3 -> ScalarVector(ArrayBuffer(32.69453124999999, 22.137890624999997)), 4 -> Scal...

entries total 
 
 
 
 
 
 
 
 
 entries total

# Compare algorithm between ParArray and Seq container

In [ ]:
val t1 = System.nanoTime
KMeans.run(parData, k, metric1, iterMax, epsilon)
val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime
KMeans.run(rawData, k, metric1, iterMax, epsilon)
val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 44106957817179
t2: Long = 44106981035837
seqData: scala.collection.mutable.Seq[org.clustering4ever.clusterizables.EasyClusterizable[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = ArraySeq(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@346f2bf,Vector()), EasyClusterizable(1,Vectorizable(ScalarVector(ArrayBuffer(14.9, 27.55))),ScalarVector(ArrayBuffer(14.9, 27.55)),shapeless.HMap@13f8a1f6,Vector()), EasyClusterizable(2,Vectorizable(ScalarVector(ArrayBuffer(14.45, 28.35))),ScalarVector(ArrayBuffer(14.45, 28.35)),shapeless.HMap@3c364393,Vector()), EasyClusterizable(...

0.41350593992124784

## Different ways to apply model to datasets

In [ ]:
val clusterized1 = model.centerPredict(parData.map(_.v))
val clusterized2 = model.obtainClustering(parData)
val clusterized3 = model.centerPredict(parData)

clusterized1: scala.collection.parallel.mutable.ParSeq[(model.ClusterID, org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]])] = ParArray((3,ScalarVector(ArrayBuffer(15.55, 28.65))), (3,ScalarVector(ArrayBuffer(14.9, 27.55))), (3,ScalarVector(ArrayBuffer(14.45, 28.35))), (3,ScalarVector(ArrayBuffer(14.15, 28.8))), (3,ScalarVector(ArrayBuffer(13.75, 28.05))), (3,ScalarVector(ArrayBuffer(13.35, 28.45))), (3,ScalarVector(ArrayBuffer(13.0, 29.15))), (3,ScalarVector(ArrayBuffer(13.45, 27.5))), (3,ScalarVector(ArrayBuffer(13.6, 26.5))), (3,ScalarVector(ArrayBuffer(12.8, 27.35))), (3,ScalarVector(ArrayBuffer(12.4, 27.85))), (3,ScalarVector(ArrayBuffer(12.3, 28.4))), (3,ScalarVector(ArrayBuffer(12.2, 28.65))), (3,ScalarVector(ArrayBuffer(13.4, 25.1))), (3,Scal...

## Plot clustering results

In [ ]:
val rawData = clusterized3.map( cz => cz.v.vector.toArray ).toArray
val labels = clusterized3.map( cz => cz.clusterIDs(0) ).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray

val (_, trueLabelsFrom0) = ExternalIndicesLocal.prepareLabels(trueLabels)

val predicts = clusterized23.map(_._1).toParArray

val nmi = ExternalIndicesLocal.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

trueLabels: scala.collection.parallel.mutable.ParArray[Int] = ParArray(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...

0.845829394276058